In [1]:

import sscps
from simple_dmet import *
from pyscf.gto import mole
from pyscf import scf, lo, fci, ao2mo
import numpy as np


def make_Hring(l=1.):
    r = l/(2 * np.sin(np.pi / 10))
    atmlst = []
    for i in range(10):
        atmlst.append(['H', (r*np.cos(2. * np.pi/10*i), r*np.sin(2. * np.pi/10*i), 0)])
    mol = gto.Mole()
    mol.atom = atmlst
    mol.basis = 'sto-6g'
    mol.build()
    return mol


def calc_ds(mol:gto.Mole):

    m = scf.RHF(mol); m.kernel()
    cal = lo.orth_ao(mol)
    clm = np.dot(np.linalg.inv(cal), m.mo_coeff)[:,np.argsort(m.mo_energy)[:mol.nelec[0]]]

    
    
    fraglsts = [[0,1],[2,3],[4,5],[6,7],[8,9]]

    p = []
    badicts = []

    dmetscf = DMET_SCF(m, [0,1], mu_glob=0.05)

    cle_frag = np.dot(np.linalg.inv(cal), dmetscf.frag_coeff)
    
    na = cle_frag.shape[1]
    cle_frag = cle_frag[:na]

    cel = np.zeros((10, 10))
    for i in range(5):
        cel[2*i:2*i+2,2*i:2*i+2] = cle_frag.conj().T

    cem = np.dot(cel, clm)

    ci_slater = sscps.ci_slater(mol.nao, mol.nelec, cem)

    def f(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.rhf()
        return nx-2
    mu_glob, count = secant(f, 0, 0.1, 1E-12)

    dmetscf.change_mu_glob(mu_glob)
    e, nx = dmetscf.rhf()
    mo_e, mo_c = dmetscf.mo_energy, dmetscf.mo_coeff
    c = mo_c[:,np.argsort(mo_e)[:2]]
    print(c)
    coef_hf = sscps.ci_slater(4, [2,2], c)
    print("coef_hf = ", coef_hf)
        
    def g(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.fci()
        return nx-2
    mu_glob, count = secant(g, 0, 0.1, 1E-12)

    dmetscf.change_mu_glob(mu_glob)
    cis = fci.direct_spin1.FCISolver(mol)
    e, coef_fci = cis.kernel(dmetscf.get_hcore(), dmetscf.erifb, 4, [2,2])

    proj, badict = sscps.get_proj(coef_hf, coef_fci, 2) # coef_fci problem???

    p = [proj] * 5
    badicts = [badict] * 5

    ci = sscps.ci_cps(mol.nao, mol.nelec, ci_slater, p, fraglsts, badicts)
    
    cae = np.dot(cal, np.linalg.inv(cel))
    eri = ao2mo.kernel(mol, cae)
    h1e = np.einsum("mj,mn,nk->jk", cae.conj(), scf.hf.get_hcore(mol), cae)

    e = fci.direct_spin1.energy(h1e, eri, ci, mol.nao, mol.nelec)

    print("Energy = ", e + mol.energy_nuc())
    return e + mol.energy_nuc()

In [2]:
bllst = np.linspace(0.5, 2.5, 21)

elst = []

for bl in bllst:
    e = calc_ds(make_Hring(bl))

    elst.append(e)

elst = np.array(elst)

np.savetxt("e_dscps_u0.txt", np.array((bllst, elst)))

converged SCF energy = -2.87405396382649


<class 'simple_dmet.DMET_SCF'> does not have attributes  bath_coeff un_dm na frag_coeff mu_glob cal un_coeff nocc dm erifb fb_coeff


converged SCF energy = -5.6276600617858
converged SCF energy = -5.68113367985478
converged SCF energy = -5.62766006178588
converged SCF energy = -5.62766006178588
[[ 9.07527850e-01 -1.22908360e-15]
 [ 4.39297160e-18 -4.19991907e-01]
 [ 6.91880431e-15  9.07527850e-01]
 [-4.19991907e-01  4.76309517e-15]]
coef_hf =  [[ 1.45278640e-01 -3.13921316e-01 -1.10909382e-15 -1.45083978e-15
   6.72330368e-02 -1.45278640e-01]
 [-3.13921316e-01  6.78328157e-01  2.39655458e-15  3.13500686e-15
  -1.45278640e-01  3.13921316e-01]
 [-1.10909382e-15  2.39655458e-15  8.46710226e-30  1.10760772e-29
  -5.13273978e-16  1.10909382e-15]
 [-1.45083978e-15  3.13500686e-15  1.10760772e-29  1.44889577e-29
  -6.71429498e-16  1.45083978e-15]
 [ 6.72330368e-02 -1.45278640e-01 -5.13273978e-16 -6.71429498e-16
   3.11145618e-02 -6.72330368e-02]
 [-1.45278640e-01  3.13921316e-01  1.10909382e-15  1.45083978e-15
  -6.72330368e-02  1.45278640e-01]]
Energy =  -2.909895868741856
converged SCF energy = -4.21118796298111
converge

In [1]:

import sscps
from simple_dmet import *
from pyscf.gto import mole
from pyscf import scf, lo, fci, ao2mo
import numpy as np
from pyscf.fci.direct_spin1 import absorb_h1e, contract_2e

def ovlp(h1e, eri, ci, ciinv, norb, nelec, link_index=None):
    h2e = absorb_h1e(h1e, eri, norb, nelec, .5)
    ci1 = contract_2e(h2e, ci, norb, nelec, link_index)
    return np.dot(ciinv.reshape(-1), ci1.reshape(-1))



def make_Hring(l=1.):
    r = l/(2 * np.sin(np.pi / 10))
    atmlst = []
    for i in range(10):
        atmlst.append(['H', (r*np.cos(2. * np.pi/10*i), r*np.sin(2. * np.pi/10*i), 0)])
    mol = gto.Mole()
    mol.atom = atmlst
    mol.basis = 'sto-6g'
    mol.build()
    return mol


def calc_ds(mol:gto.Mole):

    m = scf.RHF(mol); m.kernel()
    cal = lo.orth_ao(mol)
    clm = np.dot(np.linalg.inv(cal), m.mo_coeff)[:,np.argsort(m.mo_energy)[:mol.nelec[0]]]

    
    
    fraglsts = [[0,1],[2,3],[4,5],[6,7],[8,9]]

    p = []
    badicts = []

    dmetscf = DMET_SCF(m, [0,1], mu_glob=0.05)

    cle_frag = np.dot(np.linalg.inv(cal), dmetscf.frag_coeff)
    
    na = cle_frag.shape[1]
    cle_frag = cle_frag[:na]

    cel = np.zeros((10, 10))
    for i in range(5):
        cel[2*i:2*i+2,2*i:2*i+2] = cle_frag.conj().T

    cem = np.dot(cel, clm)

    ci_slater = sscps.ci_slater(mol.nao, mol.nelec, cem)

    def f(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.rhf()
        return nx-2
    mu_glob, count = secant(f, 0, 0.1, 1E-12)

    dmetscf.change_mu_glob(mu_glob)
    e, nx = dmetscf.rhf()
    mo_e, mo_c = dmetscf.mo_energy, dmetscf.mo_coeff
    c = mo_c[:,np.argsort(mo_e)[:2]]
    print(c)
    coef_hf = sscps.ci_slater(4, [2,2], c)
    print("coef_hf = ", coef_hf)
        
    def g(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.fci()
        return nx-2
    mu_glob, count = secant(g, 0, 0.1, 1E-12)

    dmetscf.change_mu_glob(mu_glob)
    cis = fci.direct_spin1.FCISolver(mol)
    e, coef_fci = cis.kernel(dmetscf.get_hcore(), dmetscf.erifb, 4, [2,2])

    proj, badict = sscps.get_proj(coef_hf, coef_fci, 2) # coef_fci problem???

    projinv = sscps.proj_inverse(proj, 4, [2,2], badict)

    p = [proj] * 5
    pinv = [projinv] * 5
    badicts = [badict] * 5

    ci = sscps.ci_cps(mol.nao, mol.nelec, ci_slater, p, fraglsts, badicts, normalized=False)
    ciinv = sscps.ci_cps(mol.nao, mol.nelec, ci_slater, pinv, fraglsts, badicts, normalized=False)
    
    cae = np.dot(cal, np.linalg.inv(cel))
    eri = ao2mo.kernel(mol, cae)
    h1e = np.einsum("mj,mn,nk->jk", cae.conj(), scf.hf.get_hcore(mol), cae)

    e = ovlp(h1e, eri, ci, ciinv, mol.nao, mol.nelec)
        
    print("Energy = ", e + mol.energy_nuc())
    return e + mol.energy_nuc()

In [2]:
bllst = np.linspace(0.5, 2.5, 21)

elst = []

for bl in bllst:
    e = calc_ds(make_Hring(bl))

    elst.append(e)

elst = np.array(elst)


np.savetxt("e_2H-st.txt", np.array((bllst, elst)))

converged SCF energy = -2.87405396382649


<class 'simple_dmet.DMET_SCF'> does not have attributes  fb_coeff erifb bath_coeff na cal mu_glob un_dm nocc frag_coeff dm un_coeff


converged SCF energy = -5.62766006178579
converged SCF energy = -5.68113367985468
converged SCF energy = -5.62766006178599
converged SCF energy = -5.62766006178599
[[ 9.07527850e-01  6.35962657e-15]
 [ 3.83820072e-15 -4.19991907e-01]
 [-2.59254868e-15  9.07527850e-01]
 [-4.19991907e-01 -4.47392817e-16]]
coef_hf =  [[ 1.45278640e-01 -3.13921316e-01 -9.12645352e-16 -8.63303270e-16
   6.72330368e-02 -1.45278640e-01]
 [-3.13921316e-01  6.78328157e-01  1.97206436e-15  1.86544490e-15
  -1.45278640e-01  3.13921316e-01]
 [-9.12645352e-16  1.97206436e-15  5.73326909e-30  5.42330045e-30
  -4.22360220e-16  9.12645352e-16]
 [-8.63303270e-16  1.86544490e-15  5.42330045e-30  5.13009024e-30
  -3.99525356e-16  8.63303270e-16]
 [ 6.72330368e-02 -1.45278640e-01 -4.22360220e-16 -3.99525356e-16
   3.11145618e-02 -6.72330368e-02]
 [-1.45278640e-01  3.13921316e-01  9.12645352e-16  8.63303270e-16
  -6.72330368e-02  1.45278640e-01]]
Energy =  -5.473663563749373
converged SCF energy = -4.21118796298112
converg